In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from keras.layers import BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model


In [2]:
img_size = 48
batch_size = 64

base_path = "C:\\Users\\arvin\\OneDrive\\Desktop\\miniproj\\"

# Data generator to augment data for training
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory(base_path+"train", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True)

# Data generator to augment data for validation
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory(base_path+"validation", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=False)


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [5]:
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import RMSprop

# define a new model
model2 = Sequential()

# Conv Block 1
model2.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))

# Conv Block 2
model2.add(Conv2D(128,(5,5), padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))

# Conv Block 3
model2.add(Conv2D(512,(3,3), padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))

# Conv Block 4
model2.add(Conv2D(512,(3,3), padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))

# Add a GlobalAveragePooling2D layer
model2.add(GlobalAveragePooling2D())

# Fully connected Block 1
model2.add(Dense(256))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

# Fully connected Block 2
model2.add(Dense(512))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

model2.add(Dense(7, activation='softmax'))

# Use the RMSprop optimizer with a learning rate of 0.001
opt2 = RMSprop(learning_rate=0.001)
model2.compile(optimizer=opt2, loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_12 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 activation_12 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 128)      

In [7]:
checkpoint = ModelCheckpoint("model3_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.00001,model='auto')

callbacks = [checkpoint, reduce_lr]

# Train the model for 50 epochs
epochs = 20
steps_per_epoch= train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size
history = model2.fit(
        x= train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks
)

model_json = model.to_json()
with open("model3.json","w") as json_file:
    json_file.write(model_json)

# Evaluate the model on the test set
test_loss, test_acc = model2.evaluate(validation_generator)
print('Test accuracy:', test_acc)

Epoch 1/20
450/450 [==============================] - ETA: 0s - loss: 1.5948 - accuracy: 0.3889
Epoch 1: saving model to model3_weights.h5
450/450 [==============================] - 367s 815ms/step - loss: 1.5948 - accuracy: 0.3889 - val_loss: 1.4875 - val_accuracy: 0.4331 - lr: 0.0010
Epoch 2/20
450/450 [==============================] - ETA: 0s - loss: 1.3645 - accuracy: 0.4792
Epoch 2: saving model to model3_weights.h5
450/450 [==============================] - 275s 612ms/step - loss: 1.3645 - accuracy: 0.4792 - val_loss: 1.8691 - val_accuracy: 0.3521 - lr: 0.0010
Epoch 3/20
450/450 [==============================] - ETA: 0s - loss: 1.2685 - accuracy: 0.5167
Epoch 3: saving model to model3_weights.h5
450/450 [==============================] - 278s 617ms/step - loss: 1.2685 - accuracy: 0.5167 - val_loss: 1.6376 - val_accuracy: 0.3886 - lr: 0.0010
Epoch 4/20
450/450 [==============================] - ETA: 0s - loss: 1.1567 - accuracy: 0.5588
Epoch 4: saving model to model3_weights.h5


NameError: name 'model' is not defined

In [8]:
model_json = model2.to_json()
with open("model3.json","w") as json_file:
    json_file.write(model_json)

# Evaluate the model on the test set
test_loss, test_acc = model2.evaluate(validation_generator)
print('Test accuracy:', test_acc)

111/111 [==============================] - 20s 182ms/step - loss: 1.0360 - accuracy: 0.6112
Test accuracy: 0.6112369298934937
